In [ ]:
# !pip3 install qwen-vl-utils torch torchvision jupyter gradio accelerate

In [1]:
import torch

from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

##### Load the Model

In [21]:
MODEL_PATH = "/common/public/Qwen2.5-VL/Qwen2.5-VL-7B-Instruct"

In [23]:
# default: Load the model on the available device(s)
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    MODEL_PATH, dtype=torch.bfloat16, device_map="auto"
)

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
# model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
#     MODEL_PATH,
#     dtype=torch.bfloat16,
#     attn_implementation="flash_attention_2",
#     device_map="auto",
# )


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

##### Prepare Inputs for Inference

- The default range for the number of visual tokens per image in the model is 4-16384.
- You can set min_pixels and max_pixels according to your needs, such as a token range of 256-1280, to balance performance and cost.
- min_pixels = 256*28*28
- max_pixels = 1280*28*28
- processor = AutoProcessor.from_pretrained(MODEL_PATH, min_pixels=min_pixels, max_pixels=max_pixels)

In [25]:
# default processer
min_pixels = 256*28*28
max_pixels = 1280*28*28
processor = AutoProcessor.from_pretrained(MODEL_PATH)

##### Prepare Inputs for Inference as LLM Message

In [29]:
INPUT_PROMPT = "Describe the start of the video"
IMAGE_PATH = r"./src/test_files/images.jpeg"
VIDEO_PATH = r"./src/test_files/demo_video_pixabay.mp4"

In [27]:
### IMAGES
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": IMAGE_PATH,
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

# Preparation for inference
prompt_text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
image_inputs, video_inputs = process_vision_info(messages, )
# Pass inputs to processor
inputs = processor(
    text=[prompt_text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

In [41]:
### VIDEO
FPS = 8.0

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "video",
                "video": VIDEO_PATH,
                "max_pixels": 360 * 420,
                "fps": FPS,
            },
            {
                "type": "text", 
                "text": "Select the correct answer to what is happening in this video. A) The video shows two baboons in a zoo enclosure\nB) There are two birds in a cage"},
        ],
    }
]

# Preparation for inference
prompt_text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
image_inputs, video_inputs, video_kwargs = process_vision_info(messages, return_video_kwargs=True)
print(video_kwargs)

inputs = processor(
    text=[prompt_text],
    images=image_inputs,
    videos=video_inputs,
    # fps=FPS,
    padding=True,
    return_tensors="pt",
    **video_kwargs,
)
inputs = inputs.to("cuda")

{'do_sample_frames': False, 'fps': [7.904183728359553]}


##### Run Inference

In [42]:
# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

['A) The video shows two baboons in a zoo enclosure.']
